一个 event log，记录以分钟为单位的 user event，[starttime, endtime, event]，要求 output 另一个 log，记录每分钟有多少 active events，[time, n_events]
	•	输入是：一个 List[List[int, int, str]] 结构，例如：

    logs = [
    [1, 4, "A"],
    [2, 5, "B"],
    [7, 8, "C"]
]
	•	输出要求：统计每分钟有多少 active event。例如上面的输出是：
    [
    [1, 1],
    [2, 2],
    [3, 2],
    [4, 1],
    [5, 0],
    [7, 1],
    [8, 0]
]



In [29]:
# 两个无序list，若完全一样返回true，list中是数字或者嵌套list
list1 = [[2, 1], [3, [4, 5]]]
list2 = [[1, 2], [[5, 4], 3]]

def encode(lst):
  if isinstance(lst, list):
    encoded_elements = sorted([encode(x) for x in lst])
    return '[' + ",".join(encoded_elements) + ']'
  else:
    return str(lst)
def is_equal(list1, list2):
  return encode(list1)==encode(list2)
is_equal(list1, list2)

True

In [ ]:
# "一个长度为k的motinor最多能容纳多少两个column的pins的题目; input:
#   pins = [(1,4,""L""),(2,3,""R""),(4,8,""R""),(6,9,""L"")]
#   screen_len = 5
# output: 2"

def get_max_pins(pins, screen_len):
    max_count = 0

    # 枚举所有可能的窗口起点（包含所有 top 和 bottom）
    positions = sorted(set([top for top, _, _ in pins] ))
# 
    for start in positions:
        end = start + screen_len
        count = 0
        for top, bottom, _ in pins:
            if start <= top and bottom <= end:
                count += 1
        max_count = max(max_count, count)

    return max_count

#############################################################################################

from heapq import heappush, heappop

def count_max_pins_one_column(pins, k):
    max_pins = 0
    minHeap = []
    pins.sort(key=lambda x: (x[0], x[1]))

    for top, bot in pins:
        if not minHeap or bot - top <= k:
            heappush(minHeap, (top, bot))

        while minHeap and bot - minHeap[0][0] > k:
            heappop(minHeap)

        max_pins = max(max_pins, len(minHeap))

    return max_pins


def get_max_pins(pins, k):
    # 分成左右列
    left_pins = [(top, bot) for top, bot, col in pins if col == "L"]
    right_pins = [(top, bot) for top, bot, col in pins if col == "R"]

    left_max = count_max_pins_one_column(left_pins, k)
    right_max = count_max_pins_one_column(right_pins, k)

    return left_max + right_max

pins = [
    (1, 4, "L"),
    (2, 3, "R"),
    (4, 8, "R"),
    (6, 9, "L")
]
k = 5

get_max_pins(pins, k)


# -------------------------------
def get_max_pins(pins, k):
  max_pins = 0
  minHeap = []

  pins.sort(key = lambda x: (x[0], x[1]))

  for top, bot, _ in pins:
    if not minHeap or bot-top <= k:
      heappush(minHeap, (top, bot))
      print('step1', minHeap)
    
    while minHeap and bot-minHeap[0][0] > k:
      heappop(minHeap)
      print('step2', minHeap)
    
    max_pins = max(max_pins, len(minHeap))
    print('step3', max_pins)
  
  return max_pins

[(1, 4)]
1
[(1, 4), (2, 3)]
2
[(4, 8)]
2
[(4, 8), (6, 9)]
2


2

In [27]:
def function(logs):
    # 创建事件点 (时间, 变化量)
    events = []
    for s, e, _ in logs:
        events.append((s, 1))   # 开始时间增加活跃数
        events.append((e, -1))  # 结束时间减少活跃数
    
    # 排序：时间优先，+1事件先于-1处理（确保同时刻开始事件优先）
    events.sort(key=lambda x: (x[0], -x[1]))
    
    res = []
    active = 0
    prev_time = None
    
    for time, delta in events:
        # 处理上一个时间点到当前时间点的区间
        if prev_time is not None and time > prev_time:
            # 只在有活跃事件时记录区间
            if active > 0:
                for t in range(prev_time, time):
                    res.append([t, active])
        
        # 更新当前时间点的活跃数
        active += delta
        res.append([time, active])
        prev_time = time
    
    # 去重处理（相同时间点的最新状态覆盖旧状态）
    final = {}
    for t, cnt in res:
        final[t] = cnt  # 后出现的记录会覆盖之前的
    
    # 按时间排序并生成最终结果
    return sorted([[k, v] for k, v in final.items()])

# 测试用例
logs = [[1,4,"A"], [2,5,"B"], [7,8,"C"]]
print(function(logs))

[[1, 1], [2, 2], [3, 2], [4, 1], [5, 0], [7, 1], [8, 0]]


In [ ]:
"""
- Customer C shows up to bank
- N free agents, waiting to attend customers
- M customers in line ahead of C
- Want to know how long until C is attended by an agent

Notes:
- Agent i takes agent_times[i] minutes to attend any customer
- If >= 2 agents are free at the same time, the next customer goes to agent with lowest ID
  - e.g., 2 before 4

Input:
- agent_times: List[int] = minutes it takes agents to attend any customer (length = N)
- num_customers: int = number of customers ahead of C

Output:
- Amount of time it takes for C to be attended by an agent
- ID of the agent attending C

Example:

Input:
- agent_times = [2, 1, 3]
- num_customers = 6

Output: (3, 1)

t   0   1   2
--------------
0   0   1   2
1   0   3   2
2   4   5   2
3   4   6   7
"""

# M can be 0, N>0
# - If >= 2 agents are free at the same time, the next customer goes to agent with lowest ID
# using a minHeap: (available_time, agent_id, agent_time) ->
# available_time = time until the agent is available from current min T
from  heapq import heapify, heappush, heappop
import numpy as np
def help_customer(agent_times, M):
    N = len(agent_times)
    if M == 0:
        return (0, 0)
    
    minHeap = [(0, id) for id, _ in enumerate(agent_times)]
    # [(0, 0), (0, 1), (0, 2)]
    # heapify(minHeap)


    for _ in range(M):

        available_time, id = heappop(minHeap)

        heappush(minHeap, (available_time + agent_times[id], id))
    

    return minHeap[0]

agent_times = [2, 1, 3]
for M in range(8):
    print(f"{M} {help_customer(agent_times, M)}")

# N: number of agents, M
# time: O(N+M*logN)
# space: O(N)




import heapq

def customer_wait_time(N, M, T, C):
    """
    计算第 C 位客户的等待时间。

    参数：
    N: int，客服人数
    M: int，客户总数
    T: List[int]，每位客服服务一个客户所需时间
    C: int，我们要查询的客户编号（从 1 开始）

    返回：
    int，第 C 位客户的等待时间（单位：分钟）
    """
    if N == 0 or M == 0 or C == 0 or C > M:
        return 0

    # 初始化最小堆：(客服的可用时间, 客服编号, 服务时间)
    heap = [(0, i, T[i]) for i in range(N)]
    heapq.heapify(heap)

    wait_time = 0

    for customer in range(1, C + 1):
        # 取出最早可以服务客户的客服
        available_time, agent_index, serve_time = heapq.heappop(heap)

        # 当前客户的等待时间就是该客服的可用时间
        wait_time = available_time

        # 更新客服的下一次可用时间，并放回堆中
        heapq.heappush(heap, (available_time + serve_time, agent_index, serve_time))

    return wait_time


In [ ]:
# Word Search II
class TrieNode:
    def __init__(self):
        self.children = {}
        self.isWord = False ## self.word = None

class Trie:
    def __init__(self):
        self.root = TrieNode()
    
    def insert(self, word):
        node = self.root
        for c in word:
            if c not in node.children:
                node.children[c] = TrieNode()
            node = node.children[c]
        node.isWord = True  # self.word = word

    def search(self, word):
        node = self.root
        for c in word:
            if c not in node.children:
                return False 
            node = node.children[c]
        return node.isWord
    
    def startWith(self, word):
        node = self.root
        for c in word:
            if c not in node.children:
                return False 
            node = node.children[c]
        return True

     def isPrefix(self, word):
        node = self.root
        for c in word:
            if c not in node.children:
                return False 
            node = node.children[c]
        return len(node.chidlren)>0   



class Solution:
    def findWords(self, board, words):
        trie = Trie()
        for word in words:
            trie.insert(word)
        
        ROWS, COLS = len(board), len(board[0])
        res, visited = set(), set()

        def dfs(r, c, node, word):
            if node.isWord:
                res.add(word)
                # once found the word, label it as False
                node.isWord = False 
            if r<0 or c<0 or r>=ROWS or c>=COLS or (r, c) in visited or board[r][c] not in node.children:
                return 
            
            # do
            visited.add((r,c))
            word += board[r][c]

            curr = node.children[board[r][c]]

            # backtrack
            dfs(r+1, c, curr, word)
            dfs(r-1, c, curr, word)
            dfs(r, c+1, curr, word)
            dfs(r, c-1, curr, word)

            # undo
            visited.remove((r,c))
            # we don't need to undo the word since string is immutable but list or set is mutable

        for r in range(ROWS):
            for c in range(COLS):
                dfs(r, c, trie.root, "")
        return list(res)
        # time: M is the number of cells, L is the max length of words
        # O(M*3^L)
        # space: O(N), total number of letters in the dictionanry.



        